##### IMPORT MODULES 

In [6]:
from keras.models import load_model
import os
from torchvision import transforms
import numpy as np
import cv2 as cv
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet_v2 import preprocess_input
import math


2024-01-28 19:01:30.721695: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 19:01:30.785295: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 19:01:30.785337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 19:01:30.786536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 19:01:30.794892: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 19:01:30.795336: I tensorflow/core/platform/cpu_feature_guard.cc:1

##### Constants

In [7]:
class_labels = [
    "combat",
    "destroyed_buildings",
    "fire",
    "human_aid_rehabilitation",
    "military_vehicles",
]
IMG_SIZE = (90, 90)
IMAGE_ORDER = ["C", "B", "E", "D", "A"]
predicted_class_list = {}
# transform = transforms.Compose(
#     [
#         transforms.Resize(IMG_SIZE),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ]
# )


##### UTILITY FUNCTIONS

In [8]:
def predict_class(img_path):
    loaded_model = load_model("/mnt/Storage/Dataset/model.h5")
    # print(loaded_model.summary())
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    probabilities = loaded_model.predict(img_array)
    predicted_class_index = np.argmax(probabilities)
    pred = class_labels[predicted_class_index]

    return pred


In [9]:
print(predict_class("/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/ROI2.jpg"))


1/1 [==============================] - 1s 1s/step
fire


#### **Code flow**
1. Input and preprocess the Arena Image (crop, rotation, distortion constant and camera matrix).
2. Gray Scale of the Image.
3. Aruco Markers detection for ROI.
4. ROI array creation.
5. Loop through each ROI.
6. Apply threshold sweep for each ROI.
7. Get the bounding square on the original Arena Image.
8. Flatten to a square image.
9. Add a bounding rectangle on the arena image.
10. Feed the square image into the model.
11. Add the name of the class on the arena Image.
12. Display the final image for 20 seconds after looping through each ROI.

In [10]:
def add_rectangle(frame, event_list, image_cnts, frame_count):
    # cv.imwrite("/mnt/Storage Drive/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/add_rectangle_and_name.jpg",frame)
    # frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
    for i in range(0, image_cnts):
        pts = np.reshape(event_list[i], (-1, 1, 2))
        # print("pts", event_list[i])
        width = 150
        height = 150
        dstPts = [[0, 0], [width, 0], [width, height], [0, height]]
        # print("intersect_pts", pts)
        # Get the transform
        matrix = cv.getPerspectiveTransform(np.float32(pts), np.float32(dstPts))
        # Transform the image
        out = cv.warpPerspective(frame, matrix, (int(width), int(height)))
        # out = cv.flip(out, 1)
        out = cv.rotate(out, cv.ROTATE_90_COUNTERCLOCKWISE)

        image = cv.polylines(frame, [pts], 1, (0, 255, 0), 3)
        predicted_class = ""
        if frame_count < 1:
            image_path = (
                "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/Live_Images/Img_"
                + str(i)
                + ".jpg"
            )
            # if(image_cnts == 4):
            # out = cv.rotate(out, cv.ROTATE_90_CLOCKWISE)
            cv.imwrite(image_path, out)
            predicted_class = predict_class(image_path)
            print("predicted_class", predicted_class)
            predicted_class_list[IMAGE_ORDER[i]] = predicted_class

        # else:
        #     predict_class = ""
        # print("pts",pts)
        # print("pts",pts[1][0][1]).
    # text_image = np.zeros_like(image)
    # cv.putText(
    #     text_image,
    #     predicted_class_list[i],
    #     (pts[1][0][0] + 10, pts[1][0][1] + 50),
    #     cv.FONT_HERSHEY_SIMPLEX,
    #     thickness=3,
    #     fontScale=1,
    #     color=(0, 255, 0),
    # )
    # rotation_angle = -90  # degrees
    # image_center = tuple(np.array(image.shape[1::-1]) / 2)
    # rotation_mat = cv.getRotationMatrix2D(image_center, rotation_angle, 1.)

    # # Rotate the text image
    # text_image_rotated = cv.warpAffine(text_image, rotation_mat, image.shape[1::-1], flags=cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT, borderValue=(0, 0, 0))

    # # Overlay the rotated text image onto the original image
    # image = cv.add(image, text_image_rotated)

    # image = cv.resize(image, ((1980 // 2) - 20, (1080) - 50))
    # image = cv.resize(image, (1800,1000))
    return image, predicted_class_list

    # cv.imshow("Image", image)
    # print("Out", out.shape)


In [11]:
def add_event_name(image, predicted_class_list, event_list,image_cnts):
    image = cv.rotate(image, cv.ROTATE_90_COUNTERCLOCKWISE)
    image = image[500:1750, 0:1080]
    for i in range(0,image_cnts):
        image = cv.putText(image, predicted_class_list[IMAGE_ORDER[i]],(event_list[i][0][1]+100,abs(image.shape[0]-event_list[i][0][0])+130),cv.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 255, 0), thickness=3)
    image = cv.resize(image, (1000, 900))

    cv.imshow("add_event_name",image)


In [12]:
def event_square_identification(arena):
    # cv.imwrite("/mnt/Storage Drive/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/event_square_identification.jpg",arena)
    event_list = []
    arena_org = arena.copy()
    image_cnts = 0
    border_width = 5
    arena_gray = cv.cvtColor(arena, cv.COLOR_BGR2GRAY)
    # cv.imshow("gray", arena_gray)
    arena_gray = cv.GaussianBlur(arena_gray, (3, 3), 1)
    ret, thresh = cv.threshold(arena_gray, 150, 256, cv.THRESH_BINARY)
    # cv.imshow("thresh", thresh)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        approx = cv.approxPolyDP(cnt, 0.1 * cv.arcLength(cnt, True), True)
        if len(approx) == 4:
            x1, y1, w, h = cv.boundingRect(cnt)
            ratio = float(w) / h
            area = w * h
            # print("approx", approx[0])
            if ratio >= 0.9 and ratio <= 1.2 and area > 7000 and area < 9800:
                print("Area at ", image_cnts, " is ", area)
                image_cnts += 1
                list = []
                for i in range(0, 4):
                    list.append(approx[i][0])
                event_list.append(list)
    return event_list, image_cnts


In [13]:
def detect_ArUco_details(image):
    ArUco_details_dict = {}
    ArUco_corners = {}

    ##############	ADD YOUR CODE HERE	##############
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_gray = cv.GaussianBlur(image_gray, (3, 3), 1)
    dictionary = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_1000)
    parameters = cv.aruco.DetectorParameters()
    detector = cv.aruco.ArucoDetector(dictionary, parameters)
    bboxs, ids, _ = detector.detectMarkers(image)
    # image = cv2.flip(image, 1)

    if ids is not None:
        
        ArUco_corners2 = (np.int0(bboxs)).ravel()
        # print(bboxs[0][0][0][0])
        ArUco_corners = {}
        ArUco_corners_list = []
        count1 = 0
        count2 = 0
        for i in range(0, len(ArUco_corners2), +2):
            temp = []
            temp.append(ArUco_corners2[i])
            temp.append(ArUco_corners2[i + 1])
            count1 += 1
            ArUco_corners_list.append(temp)
            if count1 % 4 == 0 and count1 != 0:
                ArUco_corners[count2] = ArUco_corners_list
                count2 += 1
                ArUco_corners_list = []
        ArUco_ID = ids.ravel()
        centre = []
        for i in range(0, len(ArUco_corners)):
            sumx = 0
            sumy = 0
            for j in range(0, 4):
                sumx += ArUco_corners[i][j][0]
                sumy += ArUco_corners[i][j][1]
            lst = []
            lst.append(int(sumx / 4.0))
            lst.append(int(sumy / 4.0))
            centre.append(lst)

        slope = []
        for i in range(0, len(ArUco_corners)):
            vr = 0
            x1 = int(round((ArUco_corners[i][0][0] + ArUco_corners[i][1][0]) / 2))
            y1 = int(round((ArUco_corners[i][0][1] + ArUco_corners[i][1][1]) / 2))
            x2 = centre[i][0]
            y2 = centre[i][1]
            c = 0
            if x2 == x1 and y2 <= y1:  # 2nd Quadrant
                vr = -180
                c += 1
            elif x2 <= x1 and y2 == y1:  # 1st Quadrant
                vr = -90
                c += 1
            elif x2 >= x1 and y2 == y1:  # 3rd Quadrant
                vr = 90
                c += 1
            elif x2 == x1 and y2 >= y1:  # 4th Quadrant
                vr = 0
                c += 1
            if c == 0:
                x1 = ArUco_corners[i][0][0]
                x2 = ArUco_corners[i][1][0]
                y1 = ArUco_corners[i][0][1]
                y2 = ArUco_corners[i][1][1]
                if x1 != x2:
                    vr = int(round(math.degrees(math.atan2((y1 - y2), (x2 - x1)))))
                vr = int(vr)
                x1 = int(round((ArUco_corners[i][0][0] + ArUco_corners[i][1][0]) / 2))
                y1 = int(round((ArUco_corners[i][0][1] + ArUco_corners[i][1][1]) / 2))
                x2 = centre[i][0]
                y2 = centre[i][1]
            slope.append(vr)
        lst = []
        for i in range(0, len(ArUco_ID)):
            lst.append(centre[i])
            lst.append(int(slope[i]))
            ArUco_details_dict[int(ArUco_ID[i])] = lst
            lst = []
        ArUco_corners_copy = ArUco_corners.copy()
        ArUco_corners = {}
        for i in range(0, len(ArUco_ID)):
            ArUco_corners[int(abs(ArUco_ID[i]))] = ArUco_corners_copy[i]

    return ArUco_details_dict, ArUco_corners


In [14]:
def get_threshold(roi_image):
    flag = 0
    for l_thresh in range(100, 255):
        # roi_image_gray = cv.GaussianBlur(roi_image, (3, 3), 1)
        roi_image_gray = cv.cvtColor(roi_image_gray, cv.COLOR_BGR2GRAY)
        kernel = np.ones((3, 3), np.uint8)
        arena_gray = cv.dilate(arena_gray, kernel, iterations=2)
        arena_gray = cv.fastNlMeansDenoising(
            src=arena_gray, dst=None, h=10, templateWindowSize=7, searchWindowSize=21
        )
        # ret, thresh = cv.threshold(roi_image_gray, l_thresh, 256, cv.THRESH_BINARY)
        thresh = cv.adaptiveThreshold(
            arena_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 11, -2
        )

        contours, hierarchy = cv.findContours(
            thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE
        )
        for cnt in contours:
            approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
            if len(approx) == 4:
                x1, y1, w, h = cv.boundingRect(cnt)
                ratio = float(w) / h
                area = w * h
                if ratio >= 0.9 and ratio <= 1.2 and area > 9600 and area < 12000:
                    flag = 1
        if flag == 1:
            return l_thresh

    if flag == 0:
        return -1


In [15]:
# get_threshold(cv.imread("/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4A/ROI_21.jpg"))


In [16]:
def ROI_Images(arena_image):
    roi_images_array = []
    event_arcs = [21, 29, 30, 34, 48]
    for i in event_arcs:
        ArUco_details_dict, _ = detect_ArUco_details(arena_image)
        if ArUco_details_dict[i][1] > -100 and ArUco_details_dict[i][1] < -80:
            point1 = (ArUco_details_dict[i][0][0] + 50, ArUco_details_dict[i][0][1])
            point2 = (
                ArUco_details_dict[i][0][0] - 120,
                ArUco_details_dict[i][0][1] - 200,
            )
            top_left = (min(point1[0], point2[0]), min(point1[1], point2[1]))
            bottom_right = (max(point1[0], point2[0]), max(point1[1], point2[1]))
            roi = arena_image[
                top_left[1] : bottom_right[1], top_left[0] : bottom_right[0]
            ]
        elif ArUco_details_dict[i][1] < 10 and ArUco_details_dict[i][1] > -10:
            roi = arena_image[
                (ArUco_details_dict[i][0][0], ArUco_details_dict[i][0][1] - 50),
                (ArUco_details_dict[i][0][0] - 200, ArUco_details_dict[i][0][1] + 120),
            ]
   
        roi_images = np.zeros_like(arena_image)
        roi_images[top_left[1] : bottom_right[1], top_left[0] : bottom_right[0]] = roi
        roi_images_array.append(roi_images)
    #     cv.imshow("roi_images"+str(i), roi_images)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
# ROI_Images(
#     cv.imread("/mnt/Storage Drive/Pictures/picture_2023-12-23_23-59-36 (copy).jpg")
# )
    return roi_images_array


In [17]:
def event_square_identification(arena_image):
    pass


In [18]:
import os
import cv2 as cv
import subprocess


def get_camera_index(camera_name):
    # List all video devices
    devices = (
        subprocess.check_output(["v4l2-ctl", "--list-devices"])
        .decode("utf-8")
        .split("\n\n")
    )

    for device in devices:
        if camera_name in device:
            # Extract the video device index from the device string
            video_device = device.split("\n")[1]
            index = int(
                video_device[-1]
            )  # Assumes the device file is in the format /dev/videoX
            return index

    print(f"Could not find a camera with the name {camera_name}")
    return None


def connect_to_camera(camera_name):
    camera_index = get_camera_index(camera_name)
    if camera_index is None:
        return None

    # Connect to the camera
    camera = cv.VideoCapture(camera_index)

    if not camera.isOpened():
        print(f"Cannot open camera named {camera_name} at index {camera_index}")
        return None

    return camera


# Example usage:
cap = connect_to_camera(
    "Lenovo FHD Webcam"
)  # Change the name to connect to a different camera

if cap is not None:
    # Capture a single frame
    ret, frame = cap.read()
    while(1):
        cv.imshow("frame",frame)
        if cv.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv.destroyAllWindows()



##### Final Function

In [19]:
def task_4a_return():
    """
    Purpose:
    ---
    Only for returning the final dictionary variable

    Arguments:
    ---
    You are not allowed to define any input arguments for this function. You can
    return the dictionary from a user-defined function and just call the
    function here

    Returns:
    ---
    `identified_labels` : { dictionary }
        dictionary containing the labels of the events detected
    """
    identified_labels = {}

    ##############	ADD YOUR CODE HERE	##############

    # try:
    #     cap = cv.VideoCapture("/dev/video2")
    #     if not cap.isOpened():
    #         raise Exception("Could not open /dev/video2")
    # except:
    #     cap = cv.VideoCapture("/dev/video3")
    #     if not cap.isOpened():
    #         print("Could not open /dev/video3 either. Please check your device.")
    #         try:
    #             cap = cv.VideoCapture("/dev/video0")
    #             if not cap.isOpened():
    #                 raise Exception("Could not open /dev/video0")
    #         except:
    #             cap = cv.VideoCapture("/dev/video1")
    #             if not cap.isOpened():
    #                 print("Could not open /dev/video1 either. Please check your device.")
    # desired_width = 1920
    # desired_height = 1080
    # cap.set(cv.CAP_PROP_FRAME_WIDTH, desired_width)
    # cap.set(cv.CAP_PROP_FRAME_HEIGHT, desired_height)
    # _, f = cap.read()
    # print("detect_ArUco_details", detect_ArUco_details(f))
    # if detect_ArUco_details(f) == ({}, {}):
    #     cap.release()
    #     try:
    #         cap = cv.VideoCapture("/dev/video0")
    #         if not cap.isOpened():
    #             print("Could not open /dev/video0")
    #             raise Exception("Could not open /dev/video0")
    #     except:
    #         cap = cv.VideoCapture("/dev/video1")
    #         if not cap.isOpened():
    #             print("Could not open /dev/video1 either. Please check your device.")
    #     desired_width = 1920
    #     desired_height = 1080
    #     cap.set(cv.CAP_PROP_FRAME_WIDTH, desired_width)
    #     cap.set(cv.CAP_PROP_FRAME_HEIGHT, desired_height)
    cap = connect_to_camera("Lenovo FHD Webcam")
    frame_count = 0
    while 1:
        # Capture the video frame
        # by frame
        ret, frame = cap.read()
        cv.imshow("frme", frame)
        # frame = cv.rotate(frame.copy(), cv.ROTATE_90_COUNTERCLOCKWISE)
        event_list, image_cnts = event_square_identification(frame)

        print("event_list", event_list)
        print("image_cnts", image_cnts)
        # frame = cv.rotate(frame.copy(), cv.ROTATE_90_COUNTERCLOCKWISE)
        image, predicted_class_list = add_rectangle(
            frame, event_list, image_cnts, frame_count
        )
        add_event_name(image, predicted_class_list, event_list, image_cnts)
        predicted_class_list = dict(
            sorted(predicted_class_list.items(), key=lambda item: item[0])
        )
        print("predicted_class_list", predicted_class_list)
        image = cv.rotate(image, cv.ROTATE_90_COUNTERCLOCKWISE)
        image = cv.resize(image, (1000, 1550))

        cv.imshow("img", image[400:1300, 20:980])
        identified_labels = predicted_class_list
        frame_count += 1
        if cv.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv.destroyAllWindows()

    ##################################################
    return identified_labels


##### Main Function

In [20]:
if __name__ == "__main__":
    identified_labels = task_4a_return()
    print(identified_labels)


TypeError: cannot unpack non-iterable NoneType object